In [18]:
import sys
sys.path.append('..')

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Tasks <a id='labeling'></a>

* Изучить используемые названия переменных. Из каких частей баланса они берутся? Как они взимосвязаны?
* [Получить данные за 2012 год (example_start.py)](#1)
* [Выбрать крупную компанию, найти на ее сайте отчетность по РСБУ за 2012 год, сравнить данные с сайта и из базы отчтености Росстата.](#compare)
* [Проверить правильность составления отчетности через равенства и неравенства (например ta = tp, debt_long < tp_long).](#2)
* [Вывести список крупнейших российских компаний по объему активов и по объему продаж. Что не так с этим списком? Как его модифцировать? (задание - сначала сделать самостоятельно, затем посмотерть play.py).](#3)


*[_Пример_](https://sebastianraschka.com/Articles/2014_ipython_internal_links.html) _создания содеражания_

> ### Получить данные за 2012 год (example_start.py).<a id='1'></a>

К [содержанию](#labeling)

Используем для этого пакет ```boo``` и код из примера к нему.

In [3]:
"""Download and truncate Rosstat corporate dataset."""

from boo import download, build, read_dataframe

print("Please be prepared download and build operations "
      "can take several minutes!")

year = 2012

# download raw file from Rosstat
try:
    download(year)
except FileExistsError:
    print("Raw file already downloaded")

# create truncated version with fewer columns and good column names
try:
    build(year)
except FileExistsError:
    print("Work file already created")

# read trimmed version of data as dataframe
df = read_dataframe(year)

print(year, "dataset:", df.shape[0], "rows and", df.shape[1], "columns")


Please be prepared download and build operations can take several minutes!
Year: 2012
Raw file already downloaded
Year: 2012
Work file already created
Year: 2012
Reading processed CSV file C:\Users\Vitaly\sandbox\data\processed-2012.csv
2012 dataset: 765813 rows and 55 columns


Загрузим данные с локальной машины.

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('../data/processed-2012.csv')

У нас данные за $2012$ год по $765 813$ предприятиям разного размера, описаны 55 признаками обозначающими показатели отчетности предприятия по РСБУ

In [3]:
data.columns

Index(['ok1', 'ok2', 'ok3', 'org', 'title', 'region', 'inn', 'okpo', 'okopf',
       'okfs', 'unit', 'of', 'of_lag', 'ta_fix', 'ta_fix_lag', 'cash',
       'cash_lag', 'ta_nonfix', 'ta_nonfix_lag', 'ta', 'ta_lag', 'tp_capital',
       'tp_capital_lag', 'debt_long', 'debt_long_lag', 'tp_long',
       'tp_long_lag', 'debt_short', 'debt_short_lag', 'tp_short',
       'tp_short_lag', 'tp', 'tp_lag', 'sales', 'sales_lag', 'profit_oper',
       'profit_oper_lag', 'exp_interest', 'exp_interest_lag',
       'profit_before_tax', 'profit_before_tax_lag', 'profit_after_tax',
       'profit_after_tax_lag', 'cash_in_oper_total', 'cash_in_oper_sales',
       'paid_to_supplier', 'paid_to_worker', 'paid_interest',
       'paid_profit_tax', 'paid_other_costs', 'cf_oper', 'paid_fa_investment',
       'cf_inv', 'cf_fin', 'cf'],
      dtype='object')

In [6]:
data.shape

(765813, 55)

Пример данных:

In [7]:
data.head(3)

,ok1,ok2,ok3,org,title,region,inn,okpo,okopf,okfs,...,paid_to_supplier,paid_to_worker,paid_interest,paid_profit_tax,paid_other_costs,cf_oper,paid_fa_investment,cf_inv,cf_fin,cf
0,65,23,1,ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО,РОССИЙСКОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ПО ПРОИЗВОДСТВ...,24,2457009983,2565,47,16,...,15215,32857,0,27105,2914527,-36814,0,29792,0,-7022
1,70,20,2,ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО,ВЛАДТЕКС,33,3328100636,31029,47,16,...,0,0,0,0,0,0,0,0,0,0
2,70,20,2,Открытое акционерное общество,Корпоративные сервисные системы,31,3125008321,104082,47,16,...,7543,52345,0,0,72151,111662,187551,-90947,-18483,2232


Также, на этом этапе можно вызвать ```pandas-profiling``` для более полного объяснительного анализа.

> ### Выбрать крупную компанию, найти на ее сайте отчетность по РСБУ за 2012 год, сравнить данные с сайта и из базы отчтености Росстата. <a id='compare'></a>

К [содержанию](#labeling)

Сравним отчетность Норильского Никеля [отсюда](https://www.e-disclosure.ru/portal/files.aspx?id=564&type=3) (Интерфакс, Центр Раскрытия Корпоративной информации, сканы документов) и его отчетность из данных Росстата. 

ИНН $8401005730$

In [8]:
data[data.title.str.lower().str.contains(r'.*(норильский никель).*', regex=True, na=False)][['inn', 'title']].head(3)

,inn,title
0,2457009983,РОССИЙСКОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ПО ПРОИЗВОДСТВ...
42959,2457195151,"ГМК ""НОРИЛЬСКИЙ НИКЕЛЬ"" И ЕГО ДОЧЕРНИХ И ЗАВИС..."
49168,8401005730,"ГОРНО-МЕТАЛЛУРГИЧЕСКАЯ КОМПАНИЯ ""НОРИЛЬСКИЙ НИ..."


Баланс активов в двух источниках отчетности равен.

In [9]:
data[data.inn==8401005730].ta

49168    703027117
Name: ta, dtype: int64

Баланс пассивов в двух источниках отчетности равен.

In [10]:
data[data.inn==8401005730].tp

49168    703027117
Name: tp, dtype: int64

> ### Проверить правильность составления отчетности через равенства и неравенства (например ta = tp, debt_long < tp_long). <a id='2'></a>

К [содержанию](#labeling)

**Проверим первое неравенство**: равенство БАЛАНС (актив) и БАЛАНС (пассив).

In [11]:
if all(data.ta == data.tp):
       print('All ta equal tp')
else:
       print(f'There is {data.shape[0] - sum(data.ta == data.tp)} ta not equal to tp ')

There is 3350 ta not to equal tp 


In [12]:
data[data.ta != data.tp][['org', 'title', 'inn']].head()

,org,title,inn
242,ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО,СЕВЕРНОЕ ЛЕСОПРОМЫШЛЕННОЕ ТОВАРИЩЕСТВО-ЛЕСОЗАВ...,2900000198
375,Открытое акционерное общество,АМТ,3624000102
1150,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ,СНЕГУРОВСКОЕ,2533007878
1506,ГЕОРГИЕВСКОЕ МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ,ТЕПЛОСЕТЬ,2625002189
2101,ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО,НАУЧНО-ПРОИЗВОДСТВЕННАЯ ФИРМА АЛТАН,2202000455


**Проверим второе неравенство**: Долгосрочные заемные средства $<$ ИТОГО долгосрочных обязательств

In [13]:
if all(data.debt_long < data.tp_long):
       print('All debt_long < tp_long')
else:
       print(f'There is {data.shape[0] - sum(data.debt_long < data.tp_long)} debt_long bigger or equal to tp_long ')

There is 732043 debt_long bigger or equal to tp_long 


In [14]:
data[data.debt_long >= data.tp_long][['org', 'title', 'inn']].head()

,org,title,inn
0,ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО,РОССИЙСКОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ПО ПРОИЗВОДСТВ...,2457009983
1,ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО,ВЛАДТЕКС,3328100636
10,ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО,ВЛАДИМИРСКИЙ ЭЛЕКТРОМЕХАНИЧЕСКИЙ ЗАВОД,3327100337
19,Общество с ограниченной ответственностью,Адопт Плюс,2617010718
20,Ликвидационная комиссия закрытого акционерного...,Сормовская ТЭЦ,5263078750


> ### Вывести список крупнейших российских компаний по объему активов и по объему продаж. Что не так с этим списком? <a id='3'></a>

К [содержанию](#labeling)

Будем выводить топ-10 крупнейших компаний.

In [15]:
NUMBER_OF_LARGE_COMPANIES = 10

#### Топ-10 по продажам

In [16]:
data.loc[data.sales.nlargest(NUMBER_OF_LARGE_COMPANIES).index, :][['org', 'title']]

,org,title
288004,Открытое акционерное общество,Газпром
288028,Открытое акционерное общество,"Нефтяная компания ""Роснефть"""
380317,Закрытое акционерное общество,Тренд
372858,Общество с ограниченной ответственностью,Финансовая компания Сбербанка
288118,Открытое акционерное общество,Российские железные дороги
401928,Общество с ограниченной ответственностью,Бизнес-Финанс
315472,Общество с ограниченной ответственностью,Кредит Свисс Секьюритиз (Москва)
701759,Открытое акционерное общество,ТНК-ВР Холдинг
187475,Открытое акционерное общество,Газпром нефть
353097,Общество с ограниченной ответственностью,Коммерц Инвестментс


#### Топ-10 по активам

In [17]:
data.loc[data.ta_fix.nlargest(NUMBER_OF_LARGE_COMPANIES).index, :][['org', 'title', 'inn']]

,org,title,inn
288004,Открытое акционерное общество,Газпром,7736050003
288118,Открытое акционерное общество,Российские железные дороги,7708503727
394654,Открытое акционерное общество,РОСНЕФТЕГАЗ,7705630445
602494,Открытое акционерное общество,СУРГУТНЕФТЕГАЗ,8602060555
288028,Открытое акционерное общество,"Нефтяная компания ""Роснефть""",7706107510
289060,Государственное унитарное предприятие города М...,Московский ордена Ленина и ордена Трудового Кр...,7702038150
405359,Открытое акционерное общество,Атомный энергопромышленный комплекс,7706664260
350008,Открытое акционерное общество,Федеральная сетевая компания Единой энергетиче...,4716016979
289914,Открытое акционерное общество,Российский концерн по производству электрическ...,7721632827
411043,Государственная корпорация по атомной энергии,Росатом,7706413348


>> ### Как его модифцировать? (задание - сначала сделать самостоятельно, затем посмотерть play.py).
*_его - список крупнейших компаний_

#### Проблема1:
* много дочерних компаний (Сбербанков, Газпромов, Роснефтей, etc) и компаний интуитивно связанных между собой, например, ВЛАДТЕКС, ВЛАДТЕКССЕРВИС.

Решим её выявив регулярными выражениями компании, содержащие в наименовании (поле ```title```) одинаковые слова.

In [18]:
print(f'''Here \
{sum(data.title.str.lower().str.contains(r'.*(газпром).*', regex=True, na=False))} title contain Газпром''')

Here 376 title contain Газпром


In [19]:
print(f'''Here \
{sum(data.title.str.lower().str.contains(r'.*(роснефть).*', regex=True, na=False))} title contain Роснефть''')

Here 31 title contain Роснефть


In [20]:
print(f'''Here \
{sum(data.title.str.lower().str.contains(r'.*(росатом).*', regex=True, na=False))} title contain росатом''')

Here 10 title contain росатом


In [21]:
print(f'''Here \
{sum(data.title.str.lower().str.contains(r'.*(сбербанк).*', regex=True, na=False))} title contain сбербанк''')

Here 10 title contain сбербанк


In [22]:
data.title[data.title.str.lower().str.contains(r'.*(сбербанк).*', regex=True, na=False)]

183880                 Сбербанк-Технологии.Нойдорф
290629         Корпоративный университет Сбербанка
303343                                Сбербанк КИБ
353483                             Сбербанк России
372858               Финансовая компания Сбербанка
372874    Специализированный депозитарий Сбербанка
417880                            Сбербанк Капитал
432288                       Сбербанк - Технологии
467345                  Сбербанк-Технологии.Сибирь
696047                        Сбербанк Лизинг Норд
Name: title, dtype: object

In [23]:
print(f'''Here \
{sum(data.title.str.lower().str.contains(r'.*(внешэкономбанк).*', regex=True, na=False))} title contain внешэкономбанк''')

Here 1 title contain внешэкономбанк


Just for fun: сколько предприятий [Векторов, Трендов, Эдельвейсов](http://lurkmore.to/%D0%9E%D0%9E%D0%9E_%C2%AB%D0%92%D0%B5%D0%BA%D1%82%D0%BE%D1%80%C2%BB)

In [24]:
print(f'''Here \
{sum(data.title.str.lower().str.contains(r'.*(вектор)|(тренд)|(эдельвейс).*', regex=True, na=False))} title contain вектор''')

Here 2300 title contain вектор
